In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType

from agents.spotify.resources.resource_play_music import play_pause_music, play_specific_song


# 🔧 Ferramenta 1: Tocar música específica
def play_specific_song_tool(input: str) -> str:
    # Espera entrada como: "Shape of You de Ed Sheeran"
    try:
        import re
        match = re.search(r"(.*) de (.*)", input, re.IGNORECASE)
        if match:
            track_name = match.group(1).strip()
            artist_name = match.group(2).strip()
            return play_specific_song(artist_name=artist_name, track_name=track_name)
        return "Formato inválido. Use: 'Nome da música de Nome do artista'."
    except Exception as e:
        return f"Erro ao interpretar entrada: {e}"


# 🔧 Ferramenta 2: Play ou Pause
def play_pause_tool(action: str) -> str:
    action = action.lower().strip()
    if action in ["play", "pause"]:
        return play_pause_music(action=action)
    return "Comando inválido. Diga 'play' ou 'pause'."


# 🎯 Lista de ferramentas para o agente
tools = [
    Tool(
        name="TocarMusica",
        func=play_specific_song_tool,
        description="Usado para tocar uma música específica no Spotify. Entrada no formato: 'Shape of You de Ed Sheeran'"
    ),
    Tool(
        name="PlayPause",
        func=play_pause_tool,
        description="Usado para dar play ou pause na música atual. Entrada: 'play' ou 'pause'."
    )
]

# 🤖 Modelo Llama3 via Ollama
llm = ChatOllama(model="llama3:8b")

# 🧠 Inicializa o agente
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

if __name__ == "__main__":
    interaction_count = 0
    max_interactions = 3

    while interaction_count < max_interactions:
        user_input = input("\n🎙️ Você: ")
        if user_input.lower() in ["sair", "exit", "quit"]:
            break
        response = agent.run(user_input)
        print(f"🤖 Agente: {response}")
        interaction_count += 1

    print("\n👋 Encerrando após 3 interações.")

